In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from dask.diagnostics import ProgressBar
from dask.distributed import progress
from distributed import Client
client = Client()
client

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import math 
import pandas_profiling
%matplotlib inline

import dask
import dask.dataframe as dd
import datetime

import warnings
warnings.filterwarnings("ignore")

import os
print(os.listdir("../input/new-york-city-taxi-fare-prediction"))


In [ ]:
print('Start of Dask Read:',datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
dask_data = dd.read_csv("/kaggle/input/new-york-city-taxi-fare-prediction/train.csv",parse_dates=["pickup_datetime"]).head(10000)
print('End of Dask Read:',datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

In [ ]:
dask_data.columns

In [ ]:
with ProgressBar():
    dask_data.head(30)

In [ ]:
display(dask_data.head(2))
print('Information:')
dask_data.info()
print('Shape:')
dask_data.shape
print('Describe:')
dask_data.describe()
print('Columns:')
len(dask_data.columns) 
print('Empty Values:')
dask_data.isnull().sum()
print('Taxi fare Mean Value:')
dask_data.fare_amount.mean()

In [ ]:
# count missing values
missing_values = dask_data.isnull().sum()
missing_values

In [ ]:
dask_data['pickup_longitude']

In [ ]:
with ProgressBar():
    dask_data.head()

In [ ]:
def sphere_dist(pickup_lat, pickup_lon, dropoff_lat, dropoff_lon):
    """
    Return distance along great radius between pickup and dropoff coordinates.
    """
    #Define earth radius (km)
    R_earth = 6371
    #Convert degrees to radians
    pickup_lat, pickup_lon, dropoff_lat, dropoff_lon = map(np.radians,
                                                             [pickup_lat, pickup_lon, 
                                                              dropoff_lat, dropoff_lon])
    #Compute distances along lat, lon dimensions
    dlat = dropoff_lat - pickup_lat
    dlon = dropoff_lon - pickup_lon
    
    #Compute haversine distance
    a = np.sin(dlat/2.0)**2 + np.cos(pickup_lat) * np.cos(dropoff_lat) * np.sin(dlon/2.0)**2
    return 2 * R_earth * np.arcsin(np.sqrt(a))
    
dask_data['haversine_dist'] = sphere_dist(dask_data.pickup_latitude, dask_data.pickup_longitude, dask_data.dropoff_latitude, dask_data.dropoff_longitude)

In [ ]:
dask_data.head()

In [ ]:
dask_data[dask_data.fare_amount<100].fare_amount.hist(bins=100, figsize=(14,3))
plt.xlabel('fare $USD')
plt.title('Histogram');

In [ ]:
# dask_data['request_hour']=pd.to_datetime(dask_data['pickup_datetime'])
dask_data['request_hour']=dask_data['pickup_datetime'].dt.hour

In [ ]:
dask_data.dtypes